In [1]:
from monai.transforms import LoadImaged, EnsureChannelFirstd, Compose
from src.transforms import (
    FindCentroid,
    GetFixedROISize,
    CropToCentroidWithSize,
    PadForCrop,
    Save,
    OrthogonalSlices,
    CropToROI,
    GetZoomedROI,
    PadForZoom,
)
from pathlib import Path

def find_corresponding_scan(segmention_path):
    scan_folder = Path(segmention_path).parent.parent / "scans"
    scan_name = Path(segmention_path).name.split(".")[0][:-2] + ".nii.gz"

    return scan_folder / scan_name

pipeline = Compose([
    LoadImaged(keys=["img", "seg"]),
    EnsureChannelFirstd(keys=["img", "seg"]),
])


segmentation_path = r'D:\premium_data\lumc\monotherapy\split_segmentations\PREM_LU_009_0.nii.gz'
scan_path = find_corresponding_scan(segmentation_path)
data = {'img':scan_path, 'seg':segmentation_path}

output = pipeline(data)


In [5]:
select = Compose(
    [
        FindCentroid(),
        GetFixedROISize(10),
        PadForCrop(),
        CropToCentroidWithSize(),
    ]
)

roi = select(output)

In [6]:
roi['centroid']

array([222, 269, 522])

In [7]:
roi['roi_size']

array([12, 12, 10])

In [8]:
roi['img'].shape

(1, 12, 12, 10)

In [22]:
output['img'].affine

tensor([[ 0.7810,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.7810,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)